In [ ]:
## Adj : Adjacency list
## Label : Label representing connected component of node 
## Size : Size of connected component of node 
## Bad : Set of labels where label : represents the component which is not bipartite
## Colour : Color map representing partition of node
## P : Partition sets  P0, P1
## V : stores visited nodes during dfs
 
class Graph:
  def __init__(self):
    self.Adj = {}
    self.Label = {}
    self.Size = {}
    self.Colour = {}
    self.P = { 0:set(), 1:set() }
    self.Bad  = set()
    self.V = set()

  def dfs_lbl(self, x, p):
    self.Label[x] = p
    if p not in self.Size:
      self.Size[p] = 1
    else:
      self.Size[p] += 1
    self.V.add(x)
    for node in self.Adj[x]:
      if node not in self.V:
        self.dfs_lbl(node, p)

  def dfs_bp(self, x, clr):
    self.Colour[x] = clr
    self.V.add(x)
    if x not in self.P[clr]:
      self.P[clr^1].remove(x)
      self.P[clr].add(x)
    for node in self.Adj[x]:
      if node == x:
        self.Bad.add(self.Label[x])
        return True
      if node not in self.V:
        if self.dfs_bp(node, clr^1):
          self.Bad.add(self.Label[x])
          return True
      else:
        if self.Colour[x] == self.Colour[node]:
          self.Bad.add(self.Label[x])
          return True  
    return False
 
  def add_node(self, x):
    assert x not in self.Adj , "Node {} is already in the graph".format(x)
 
    self.Adj[x] = {}
    self.Label[x] = x
    self.Size[x] = 1
    self.Colour[x] = 0
    self.P[0].add(x)
 
  def add_edge(self, u,v):
    assert u in self.Adj , "Node {} not found in the graph".format(u)
    assert v in self.Adj , "Node {} not found in the graph".format(v)
 
 
    if self.Label[u] == self.Label[v]:
      if self.Colour[u] == self.Colour[v]:
        self.Bad.add(self.Label[u])
        if u in self.Adj[v]:
          self.Adj[u][v] += 1
          self.Adj[v][u] += 1
        elif u == v:
          self.Adj[u][u] = 2 
        else:
          self.Adj[u][v] = 1
          self.Adj[v][u] = 1
        return
    
    if self.Size[self.Label[u]] > self.Size[self.Label[v]]:
        u,v = v,u

    bipartite = not ((self.Label[u] in self.Bad) or (self.Label[v] in self.Bad))

    if bipartite:
      if self.Colour[u] == self.Colour[v]:
          self.V = set()
          self.dfs_bp(u, self.Colour[u]^1)

    del self.Size[self.Label[u]]
    del self.Label[u]

    self.V = set()
    self.dfs_lbl(u,self.Label[v])

      
  def remove_node(self, x):
    assert x in self.Adj , "Node {} is not in the graph".format(x)
 
    adj = list(self.Adj[x].keys())
 
    if x in adj:
      adj.remove(x)
 
    for node in adj:
      del self.Adj[node][x] 
    
    del self.Adj[x]
    del self.Size[self.Label[x]]
    
    if self.Label[x] in self.Bad:
      self.Bad.remove(self.Label[x])
      bipartite = False
    else:
      bipartite = True
 
    self.P[self.Colour[x]].remove(x)
    del self.Colour[x]

    self.V = set()
    for node in adj:
      if node not in self.V:
        self.dfs_lbl(node,node)
 
    if not bipartite :
      self.V = set()
      for node in adj:
        if node not in self.V:
          self.dfs_bp(node,self.Colour[node])
      
      
  def remove_edge(self, u,v):
    assert u in self.Adj , "Node {} not found in the graph".format(u)
    assert v in self.Adj , "Node {} not found in the graph".format(v)
    assert v in self.Adj[u] , "Edge {} {} not found in the graph".format(u,v)
 
    self.Adj[u][v] -= 1
    self.Adj[v][u] -= 1
 
    if self.Adj[u][v] == 0:
 
      del self.Adj[u][v]

      if u != v :
        del self.Adj[v][u]

      bipartite = self.Label[u] not in self.Bad  

      if not bipartite :      
        self.Bad.remove(self.Label[u])
 
      del self.Size[self.Label[u]]
      del self.Label[u]
 
      self.V = set()
      self.dfs_lbl(u, u)

      if not bipartite :
        self.V = set()
        self.dfs_bp(u, self.Colour[u])
 
      if v not in self.V:
        self.V = set()
        self.dfs_lbl(v, v)

        if not bipartite :
          self.V = set()
          self.dfs_bp(v, self.Colour[v])
  
  def is_bipartite(self):
    return len(self.Bad) == 0

  def print_bipartite(self):
    if not self.is_bipartite() :
      return print("Graph is not bipartite!!\n")
    print("A :",self.P[0])
    print("B :",self.P[1],'\n')


In [ ]:
G = Graph()

for i in range(1,11):
  G.add_node(i)

G.add_edge(5, 10)
G.add_edge(6, 8)
G.add_edge(1, 2)
G.add_edge(1, 4)
G.add_edge(1, 6)
G.add_edge(3, 4)
G.add_edge(3, 6)
G.add_edge(5, 6)
G.add_edge(6, 9)
G.add_edge(7, 8)
G.add_edge(8, 9)

print("Op ",1)
# initially not bipartite
G.print_bipartite()

print("Op ",2)
# remove edge to make bipartite
G.remove_edge(8, 9)
G.print_bipartite()

print("Op ",3)
# check loop condition
G.add_edge(9, 9)
G.print_bipartite()
print()

print("Op ",4)
# remove loop
G.remove_edge(9, 9)
G.print_bipartite()
print()

print("Op ",5)
# add edge to remove bipartiteness
G.add_edge(6,10)
G.print_bipartite()


print("Op ",6)
# remove node and check bipartiteness
G.remove_node(5)
G.print_bipartite()
